In [1]:
#some libraries
!pip install -qU \
  sentence-transformers==2.2.2 \
  accelerate==0.21.0 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 10.6 MB/s eta 0:00:00
   

In [2]:
#specific version to avoid error with vector store
!pip install chromadb==0.5.3
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
#update transformers
!pip install -U transformers

In [4]:
#loading the llama-2-7b-chat model and the tokenizer
from torch import cuda, bfloat16
import transformers

model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = 'hf_token'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")
#tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:961: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

Model loaded on cuda:0


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
#sample prompt
#prompt = "From the following sentence, please extract the optical power of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text:Operating at 5 K in pulsed mode, the threshold current density is 840 A/cm2, and the peak power is approximately 2.5 mW."
prompt=  "From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: Employing a scattering-assisted injection and resonant-phonon depopulation method, a QCL operating at 17.3 THz frequency has been successfully developed."
prompt = f"[INST]{prompt} [/INST]"

In [7]:
#view prompt
print(prompt)

[INST]From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: Employing a scattering-assisted injection and resonant-phonon depopulation method, a QCL operating at 17.3 THz frequency has been successfully developed. [/INST]


In [8]:
#sample output with pipeline-no RAG
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800,)
result = pipe(prompt)
print(result[0]['generated_text'][len(prompt):])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 The design type of the Quantum Cascade Semiconductor laser device, as given in the input text, is not explicitly mentioned. However, it can be inferred that it is a Quantum Cascade Laser (QCL) due to the term "Quantum Cascade Semiconductor laser." The specific design details, such as the number of quantum wells, active layer structure, etc., are not provided in the text.


In [9]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#loading training data for providing context
import pandas as pd
train_data_df=pd.read_csv('/content/drive/MyDrive/train_data_split.csv')
## Convert the DataFrame to a format compatible with Hugging Face Datasets
train_data_df['Instruction'] = train_data_df['Instruction']+ ". Input Text: " + train_data_df['input_text'] + "\n\nExtracted Properties:\n\n"+ train_data_df['Output'].astype(str)
train_dataset_dict = {
    'instruction': train_data_df['Instruction'].tolist()
}

In [11]:
#verify train data dictionary
#train_dataset_dict

In [12]:
#creating context document from the list
docs=train_dataset_dict['instruction']

In [13]:
#initializing the embedding model
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder = '/data/base_models')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
#do the embedding
embeddings = embedding_model.encode(docs)
#verify the embeddings
embeddings.shape

(1040, 768)

In [15]:
#indexing-run this only once
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="q-rag_mistral-7b-instruct")

In [16]:
#addding the train data
collection.add(
    embeddings = embeddings,
    documents=docs,
    ids= [str(i) for i in range(len(docs))]
)

In [17]:
#retrieving
def retrieve_vector_db(query, n_results=3):
    results = collection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=n_results
    )
    return results['documents']

In [18]:
#sample query
'''
#we can have different queries depending on the property we are interested in
#query=input ("Please Enter your query here: ")
query = "From the following sentence, please extract the heterostructure of the Quantum Cascade Semiconductor laser device: "
#input_text is the user input text containing the properties for extraction
#print("Enter the text containing the properties herer")
input_text= "We report the development of a quantum cascade laser, at l587.2 mm, corresponding to 3.44 THz or 14.2 meV photon energy. The GaAs/Al0.15Ga0.85As laser structure utilizes longitudina loptical (LO) phonon scattering for electron depopulation. Laser action is obtained in pulsed mode at temperatures up to 65 K, and at 50 % duty cycle up to 29 K. Operating at 5 K in pulsed mode, the threshold current density is 840 A/cm2, and the peak power is approximately 2.5 mW. Based on the relatively high operating temperatures and duty cycles, we propose that direct LO-phonon-based depopulation is a robust method for achieving quantum cascade lasers at long-wavelength THz frequencies."
query= query + input_text
'''
query=prompt
retrieved_results = retrieve_vector_db(query)
print(query)

[INST]From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: Employing a scattering-assisted injection and resonant-phonon depopulation method, a QCL operating at 17.3 THz frequency has been successfully developed. [/INST]


In [23]:
#viewing the retreived 3 sample context text based on the user query.
retrieved_results[0]

['From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text:  A QCL with a 2.1 THz frequency was shown to use a scattering-assisted injection and resonant-phonon depopulation design in its operation.\t\t\n\nExtracted Properties:\n\nresonant-phonon',
 'From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text: A design scheme involving scattering-assisted injection and resonant-phonon depopulation was employed in the demonstration of a 2.1 THz quantum cascade laser (QCL).\t\n\nExtracted Properties:\n\nresonant-phonon',
 'From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text:  A QCL operating at a frequency of 17.3 THz has b

In [24]:
#assigning the context
context=retrieved_results[0]

In [25]:
#prompt template
prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations. The output format should correspond to the one in the example sentences. Example sentences containing an instruction, input text and the extracted properties are given below:

Example Sentences: {context}

Instruction: {query}

[/INST]
'''

In [26]:
#view prompt
print(prompt)


[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations. The output format should correspond to the one in the example sentences. Example sentences containing an instruction, input text and the extracted properties are given below:

Example Sentences: ['From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text:  A QCL with a 2.1 THz frequency was shown to use a scattering-assisted injection and resonant-phonon depopulation design in its operation.\t\t\n\nExtracted Properties:\n\nresonant-phonon', 'From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text. Input Text: A design scheme i

In [27]:
#sample output with transformer pipeline
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800,)
result = pipe(prompt)
print(result[0]['generated_text'][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Extracted Properties:

resonant-phonon

'''
